# Import software libraries and load the dataset #

In [1]:
import sys                                             # Read system parameters.
import numpy as np                                     # Work with multi-dimensional arrays and matrices.
import pandas as pd                                    # Manipulate and analyze data.
import sklearn                                         # Perform data mining and analysis.
from sklearn import datasets
from time import time                                  # Calculate training time.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- scikit-learn {}\n'.format(sklearn.__version__))

# Load the dataset.
boston = datasets.load_boston()
print('Loaded {} records.'.format(len(boston.data)))

# Convert array to pandas DataFrame.
data_raw = pd.DataFrame(boston['data'], columns = boston['feature_names'])
data_raw['target'] = boston['target']

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- NumPy 1.16.2
- pandas 0.24.2
- scikit-learn 0.20.3

Loaded 506 records.


# Split the datasets

In [2]:
from sklearn.model_selection import train_test_split

# 'target' is the dependent variable (value to be predicted), so it will be
# removed from the training data and put into a separate DataFrame for labels.
label_columns = ['target']

# Split the training and test datasets and their labels.
X_train, X_test, y_train, y_test = train_test_split(data_raw.loc[:, 'CRIM': 'LSTAT'],
                                                                            data_raw[label_columns],
                                                                            random_state = 2)

# Compare the number of rows and columns in the original data to the training and test sets.
print(f'Original set:        {data_raw.shape}')
print('------------------------------')
print(f'Training features:   {X_train.shape}')
print(f'Test features:       {X_test.shape}')
print(f'Training labels:     {y_train.shape}')
print(f'Test labels:         {y_test.shape}')

Original set:        (506, 14)
------------------------------
Training features:   (379, 13)
Test features:       (127, 13)
Training labels:     (379, 1)
Test labels:         (127, 1)


# Drop columns that won't be used for training

In [3]:
# Drop column from dataset that shows weak correlation.
def drop_unused(dataset):
    
    print('Columns before drop:\n\n{}\n'.format(list(dataset.columns)))
        
    dataset = dataset.drop(['CHAS'], axis = 1)
    
    print('Columns after drop:\n\n{}\n'.format(list(dataset.columns)))
    return dataset

X_train, X_test = drop_unused(X_train.copy()), drop_unused(X_test.copy())

Columns before drop:

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

Columns after drop:

['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

Columns before drop:

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

Columns after drop:

['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']



# Standardize the features

In [4]:
def standardize(X):
    result = X.copy()
    
    for feature in X.columns:
        result[feature] = (X[feature] - X[feature].mean()) / X[feature].std()  # z-score formula.
        
    return result

X_train = standardize(X_train)
X_test = standardize(X_test)

print('The features have been standardized.')

The features have been standardized.


# Train a model and calculate its cost

In [5]:
from sklearn.metrics import mean_squared_error as mse

def model_train(model):
    start = time()
    model.fit(X_train, np.ravel(y_train))
    end = time()
    train_time = (end - start) * 1000
    
    predict = model.predict(X_test)
    
    cost = mse(y_test, predict)
    
    print('Linear regression model took {:.2f} milliseconds to fit.'.format(train_time))
    print('Cost (mean squared error): {:.2f}'.format(cost))
    
print('The function to train the model and calculate its cost has been defined.')

The function to train the model and calculate its cost has been defined.


# Evaluate linear regression models using both closed-form and iterative solutions

In [6]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor

# Create closed-form and iterative ridge regression models.
def model_eval(eta):
    for name, model in [
        ('Ridge regression (closed form)', Ridge(alpha = 0.1, solver = 'cholesky')),
        ('Ridge regression (gradient descent)', SGDRegressor(penalty = 'l2',
                                                             alpha = 0.1,
                                                             tol = 1e-3,
                                                             learning_rate = 'constant',
                                                             eta0 = eta,
                                                             random_state = 2))]:

        print('Model: {}'.format(name))
        print('--------------------')
        model_train(model)
        print('\n')
        
print('The function to evaluate the linear regression models has been defined.')

The function to evaluate the linear regression models has been defined.


In [7]:
model_eval(0.09)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 1.80 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 1.40 milliseconds to fit.
Cost (mean squared error): 65.36




In [8]:
model_eval(0.08)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 1.21 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 1.08 milliseconds to fit.
Cost (mean squared error): 47.05




In [9]:
model_eval(0.05)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 1.20 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 0.96 milliseconds to fit.
Cost (mean squared error): 31.89




In [10]:
model_eval(0.01)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 1.17 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 1.32 milliseconds to fit.
Cost (mean squared error): 25.25


